In [17]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [18]:
import os,sys
#sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib widget
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import Layout
from pathlib import Path
from scipy import spatial
from IPython.display import display_markdown

from ipywidgets.widgets.interaction import show_inline_matplotlib_plots

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

%use gromacs-5.1.4
%use pymol-1.8.4

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)


In [19]:
import time 
def mutate_residue(mdstruc, resnumber, mutresname):
    
    deletion = False
    resndx = -1
    chain = ''
    for r in mdstruc.residues:
        if(r.number==resnumber):
            
            foundC = False
            foundO = False
            foundN = False
            foundCA = False
            for at in r:    
                if at.name=='C':
                    foundC = True
                elif at.name=='O':
                    foundO = True
                elif at.name=='N':
                    foundN = True
                elif at.name=='CA':
                    foundCA = True
                
            if foundC * foundN * foundO * foundCA:
                chain = r.chain
                resndx = r.idx
            else:
                print_output('Could not locate necessary backbone atoms: Eliminating from the structure.')
                mdstruc.strip(':'+str(r.idx+1))
                deletion = True
            break
            
    if resndx>=0:
        r = mdstruc.residues[resndx]
        print_output('Attempting to repair using PyMol mutagenesis wizard')
        
        # First replace with Gly: sometimes PyMol gets confused by incomplete side chains.
        mdstruc.strip('(:'+str(r.idx+1)+')&!(@C,CA,N,O)')
        mdstruc.residues[r.idx].name = 'GLY'
        mdstruc.write_pdb('gmx/tmp/tmp.pdb', altlocs='first')
        out = !{'pymol -qc misc/mutate.py -- gmx/tmp/tmp.pdb ' + chain + '/' + str(resnumber) + '/ ' + mutresname}
        print_log(out)
                
        # Check for errors
        error = False
        for line in out:
            if line.find('Error')>=0:
                error = True
                break
        if error:
            if(error):
                print_output('PyMol Error. (See logs.) Replaced with GLY.')
                
        else:
            print_output("Repair successful.")
            mdstruc = pmd.load_file('gmx/tmp/tmp.pdb.mut')
        
    return mdstruc, deletion
    
def modlog(fname, text):
    with open("gmx/tmp/" + fname+"_log", "a") as fd:
        fd.write(text + "\n")
        fd.close()
        
def run_pdb2gmx(mdstruc, fname, lastPass):
    
    bondmax = 1.5*2
    
    success = True
    MissingAtomRegEx = re.compile('.*Residue(.*)named(.*)of a molecule in the input file was mapped to an entry in the topology database, but the atom(.*)used in that entry is not found in the input file')
    ResMisMatchRegEx = re.compile('.*Atom(.*)in residue[ ]+([a-zA-Z]+)[ ]+([0-9]+)[ ]+was not found in rtp entry(.*)with(.*)atoms while sorting atoms')
    ResNotFoundRegEx = re.compile('.*Residue(.*)not found in residue topology database')
    
    # Clean output directory before starting. 
    out = !{"rm gmx/tmp/*.gro*"}
    print_log(out)
    
    out = !{"rm gmx/tmp/*.top*"}
    print_log(out)
    
    out = !{"rm gmx/tmp/*.itp*"}
    print_log(out)
    
    out = !{"rm gmx/tmp/*.pdb.*"}
    print_log(out)
    
    out=!{"cd gmx/tmp/; export GMX_MAXBACKUP=-1; gmx pdb2gmx -ff "+ ffselect.value + " -f " + fname + " -o input.gro -ignh -water " + waterselect.value}
    print_log(out)
    msg = ''
    
    madeTop = False
    MadeTopRegEx = re.compile('You have successfully generated a topology')
    for line in out:
        if re.match(MadeTopRegEx, line):
            madeTop = True
    
    WarnNonSeqChain = re.compile('.*Chain identifier \'(.*)\' is used in two non-sequential blocks.*')
    WarnOccup = re.compile('.*there were (.*) atoms with zero occupancy and (.*) atoms.*')
    WarnLongBond = re.compile('.*Long Bond.*\(([0-9]+)-([0-9]+).*\).*')
    WarnAtomMissing = re.compile('.*atom (.*) is missing in residue ([a-zA-Z]+)[ ]+([0-9]+) in the pdb file.*')

    for line in out:
        if(success and re.search('warning', line, flags=re.IGNORECASE)):
            
            ##############################################
            #  Missing atoms --> Mutate
            ##############################################
            if re.match(WarnAtomMissing, line):
                hit = re.match(WarnAtomMissing, line)
                resname = hit.group(2).strip()
                resnumber = int(hit.group(3).strip())
                print_output('Atoms were missing from residue '+resname+ ' ' + str(resnumber))
                mdstruc, delflag = mutate_residue(mdstruc, resnumber, resname)
                mdstruc.write_pdb('gmx/tmp/'+fname)
                if delflag:
                    modlog(fname, "Eliminated " + str(resnumber))
                else:
                    modlog(fname, "Mutated " + str(resnumber))
                success = False
                
            ##############################################
            #  Long bond --> Insert chain break
            ##############################################
            elif re.match(WarnLongBond, line):
                if madeTop:
                    hit = re.match(WarnLongBond, line)
                    at1tmp = int(hit.group(1))
                    at2tmp = int(hit.group(2))
                    at1 = min(at1tmp,at2tmp)
                    at2 = max(at1tmp,at2tmp)
                    
                    grostruc = pmd.load_file('gmx/tmp/input.gro')
                    
                    x1 = grostruc.coordinates[at1-1]
                    x2 = grostruc.coordinates[at2-1]
                    dist = np.linalg.norm(x1 - x2)
                    
                    # If this is an amide bond, and the distance is unreasonably large, insert a 
                    # chain break between the residues.
                    if (dist>bondmax) and (grostruc[at1-1].name=='C') and (grostruc[at2-1].name=='N'):
                        res1 = grostruc[at1-1].residue.number
                        res2 = grostruc[at2-1].residue.number
                        print_output('Bond between atoms ' + str(at1) + ' and ' + str(at2) + ' was too long (' + '{:.2f}'.format(dist) +' Ang).')
                        print_output('Identified atoms as belonging to residues ' + str(res1) + ' and ' + str(res2))
                        print_output('Inserting chain break between them.')
                        if res1<res2:
                            grostruc[at1-1].residue.ter = True
                        else:
                            grostruc[at2-1].residue.ter = True
                        grostruc.write_pdb('gmx/tmp/'+fname)
                        modlog(fname, "BrokeAfter " + str(res1))
                        success = False
            elif re.match(WarnNonSeqChain, line):
                if lastPass:
                    print_output('Non-sequential chain warning')
            elif re.match(WarnOccup, line):
                if lastPass:
                    print_output('Occupancy warning')
            else:
                if lastPass:
                    print_output(line)
    
    if success:
        for line in out:
            # If we find an error, start recording message for output
            if re.search('error', line, flags=re.IGNORECASE) and (len(msg)==0):
                success = False
                msg = line
            elif len(msg)>0:
                if len(line.strip())>0:
                    msg = msg + line + " "
                else:
                    ##############################################
                    #  Missing atoms --> Mutate
                    ##############################################
                    if re.match(MissingAtomRegEx, msg):
                        hit = re.match(MissingAtomRegEx, msg)
                        resnumber = int(hit.group(1))
                        resname = hit.group(2).strip()
                        print_output('Atoms were missing from residue '+resname+ ' ' + str(resnumber))
                        mdstruc, delflag = mutate_residue(mdstruc, resnumber, resname)
                        mdstruc.write_pdb('gmx/tmp/'+fname)
                        if delflag:
                            modlog(fname, "Eliminated " + str(resnumber))
                        else:
                            modlog(fname, "Mutated " + str(resnumber))
                            
                    ##############################################
                    #  Residue not in data base --> Eliminate 
                    ##############################################
                    elif re.match(ResNotFoundRegEx, msg):
                        hit = re.match(ResNotFoundRegEx, msg)
                        resname = hit.group(1).strip().strip('\'')
                        print_output('Residue '+resname+ ' not available in the force field database.')
                        print_output('Eliminating from the structure.')
                            
                        # Keep eliminating residues until they're all gone
                        still_more = True
                        while still_more:
                            # Find the next residue of this type
                            for r in mdstruc.residues:
                                if(r.name==resname):
                                    resnum = r.number
                                    residx = r.idx
                                    break
                                    
                            mdstruc.strip(":"+str(residx+1))
                            mdstruc.write_pdb('gmx/tmp/'+fname)
                            modlog(fname, "Eliminated " + str(resnum))
                            print_output('Eliminated residue: ' + str(resnum))
                                
                            # Reload the file so that residue numbers are always up-to-date
                            mdstruc = pmd.load_file('gmx/tmp/'+fname)
                            
                            # Check whether we need to cycle again
                            still_more = False
                            for r in mdstruc.residues:
                                if(r.name==resname):
                                    still_more = True
                        
                    
                    ##############################################
                    #  Residue doesn't match database --> Eliminate
                    ##############################################
                    elif re.match(ResMisMatchRegEx, msg):
                        hit = re.match(ResMisMatchRegEx, msg)
                        resname = hit.group(2).strip()
                        resnum = int(hit.group(3).strip())
                        print_output('Residue ' +resname+ ' ' + str(resnum) + ' did not match the force field database entry.')
                        print_output('Eliminating from the structure.')
                        for r in mdstruc.residues:
                            if(r.number==resnum):
                                mdstruc.strip(":"+str(r.idx+1))
                        mdstruc.write_pdb('gmx/tmp/'+fname)
                        modlog(fname, "Eliminated " + str(resnum))
                    else:
                        if lastPass:
                            print_output(msg)

                    msg = ''
    return success



# def mdviewgo_onclick(b):
#     global mdreps
#     for n in range(0, len(mdreps)):
#         mdview.set_representations(mdreps[n], component=n)

def ffgo_onclick(b):
#     global mdview
    global mdreps
    
#     while len(mdview._ngl_component_ids)>0:
#         mdview.remove_component(mdview._ngl_component_ids[0])
#     mdviewout.clear_output()
#     with mdviewout:
#         display(mdview)
        
    mdreps = []
    
    chainList = list()
    for r in struc.residues:
        if chainList.count(r.chain)==0:
            chainList.extend(r.chain)
            
    # First identify which chains should be displayed
    chainList = []
    # Loop through chain-selection check-boxes
    for cb in accordion.children[0].children[1:]:
        if cb.value==True:
            chainList.extend(cb.description)
    
    failed = False
    for chain in chainList:
        fname = 'input_'+ chain + '.pdb'
        struc.write_pdb('gmx/tmp/'+fname)
        mdstruc = pmd.load_file('gmx/tmp/'+fname)
        print_output("")
        print_output("********************************************")
        print_output('Preparing chain ' + str(chain) + ' for MD...')
        print_output("")
        mdstruc[chain,:,:].write_pdb('gmx/tmp/'+fname)
        mdstruc[chain,:,:].write_pdb('gmx/tmp/'+fname+'_original')
        out = !{":>gmx/tmp/" + fname + '_log'}
        print_log(out)
        
        maxTries = 100
        tries = 0
        tryAgain = True
        while tryAgain:
            tries += 1
            mdstruc = pmd.load_file('gmx/tmp/'+fname)
            mdstruc.write_pdb('gmx/tmp/' + fname + '.' + str(tries))
            if(run_pdb2gmx(mdstruc, fname, False)):
                tryAgain = False
                #run_pdb2gmx(mdstruc, fname, True)
                print_output('Chain ' + chain + ' successfully parameterized!')
                
                
            elif tries >= maxTries:
                tryAgain = False
                print_output('Failed to generate gmx input in ' + str(maxTries) + ' attempts')
                failed = True
        if failed:
            break
    if failed==False:
        fname = 'input.pdb'
        out=!{":>gmx/tmp/" + fname}
        for chain in chainList:
            out=!{'cat gmx/tmp/input_'+chain+'.pdb >> gmx/tmp/' + fname}
        
        out=!{'cat gmx/tmp/input.pdb'}
        mdstruc = pmd.load_file('gmx/tmp/'+fname)
        if(run_pdb2gmx(mdstruc, fname, True)):
            print_output('Successfully parameterized all chains!')
            solvgo.disabled = False
            with view_changes_out:
                display_markdown("<a href=\"view_modifications.ipynb\" target=\"_blank\">View Changes</a>", raw=True)
#             mdviewgo.disabled = False
            mdacc.selected_index = 0
            with open("gmx/tmp/chain_list.txt", "w") as fd:
                for chain in chainList:
                    fd.write(chain + "\n")
                fd.close()
            
#             mdview.clear(0)
#             with mdviewout:
#                 display(mdview)
            

def print_flags(output):
    error = False
    flags = ["warning", 'error', 'note']
    print_line = False
    for line in output:
        
        # If the output line contains one of the flags, start printing
        for flag in flags:
            if line.lower().find(flag)>=0:
                print_line = True
                if flag=='error':
                    error = True
                
        # If the output line is blank, stop printing. 
        if len(line.strip())==0:
            # If currently printing, print one blank line to mark end
            if print_line:
                print_output("")
            print_line = False
            
            
        # Print, if appropriate
        if print_line:
            print_output(line)
    return error
            
            
def solvgo_onclick(b):
    err = False
    print_output('')
    
    out = !{"rm gmx/solv/*.*"}
    print_log(out)
    
    out = !{"cp gmx/tmp/input.gro gmx/solv/; cp gmx/tmp/*.itp gmx/solv/; cp gmx/tmp/*.top gmx/solv/"}
    print_log(out)
    
    out = !{" cd gmx/solv/; gmx editconf -f input.gro -o conf_box.gro -c -d 0.5 -bt cubic"}
    err = print_flags(out)
    print_log(out)
    
    out = !{" cd gmx/solv/; gmx solvate -cp conf_box.gro -cs spc216.gro -o conf_solv.gro -p topol.top"}
    err = (err or print_flags(out))
    print_log(out)
    
    out = !{'cd gmx/solv/; gmx grompp -f ../mdp/ions.mdp -c conf_solv.gro -p topol.top -o ions.tpr'}
    err = (err or print_flags(out))
    print_log(out)
    
    out = !{'cd gmx/solv/; gmx genion -s ions.tpr -o conf_ions.gro -p topol.top -pname NA -nname CL -neutral'}
    err = (err or print_flags(out))
    print_log(out)
    
    if err==False:
        emgo.disabled = False
        mdacc.selected_index = 2
    
    
def emgo_onclick(b):
    print_output('')
    out = !{"rm gmx/em/*.*"}
    print_log(out)
    
    out = !{"cp gmx/solv/*.itp gmx/em/; cp gmx/solv/*.top gmx/em/"}
    print_log(out)
    
    out = !{'cd gmx/em/; gmx grompp -f ../mdp/minim.mdp -c ../solv/conf_ions.gro -p topol.top -o em.tpr'}
    err = print_flags(out)
    print_log(out)
    
    if err==False:
#         out = !submit --progress text --inputfile -v standby_S@brown gmx/em/em.tpr gmx mdrun -v -deffnm em
#         err = print_flags(out)
#         print(out)
        
        out = !{'cd gmx/em/; gmx mdrun -v -deffnm em'}
        err = print_flags(out)
        print_log(out)
    
#     if err==False:
#         !{"cd gmx/em/; echo \"10 0\\n\" | gmx energy -f em.edr -o potential.xvg"}
#         dat = np.loadtxt('gmx/em/potential.xvg', comments=["@", "#"])
#         with mainout:
#             %matplotlib widget
#             plt.plot(dat[:,0], dat[:,1])
#             #show_widget_matplotlib_plots()
#             plt.show()
    return err
        
        
ffgo = widgets.Button(
        description='Prepare Topology',
        disabled=True,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to prepare structure for molecular dynamics (MD)',
        icon='' # (FontAwesome names without the `fa-` prefix)
    )
ffgo.on_click(ffgo_onclick)

view_changes_out = widgets.Output()

# mdviewgo = widgets.Button(
#         description='View Modifications',
#         disabled=True,
#         button_style='', # 'success', 'info', 'warning', 'danger' or ''
#         tooltip='Click to view modifications',
#         icon='' # (FontAwesome names without the `fa-` prefix)
#     )
# mdviewgo.on_click(mdviewgo_onclick)

fflabel = widgets.Label(value='Force Field')
ffselect = widgets.RadioButtons(
    options=['charmm27', 'oplsaa'],
    value='charmm27',
    #description='Force field for MD',
    disabled=True
)

waterlabel = widgets.Label(value='Water Model')
waterselect = widgets.RadioButtons(
    options=['spce', 'tip3p', 'spc'],
    value='spce',
    disabled=True
)

def set_md_state(choice):
    ffgo.disabled = choice
    ffselect.disabled = choice
    waterselect.disabled = choice

ffbox = widgets.VBox([fflabel, ffselect, waterlabel, waterselect, ffgo, view_changes_out])

solvgo = widgets.Button(
        description='Solvate',
        disabled=True,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to add water and counter-ions',
        icon='' # (FontAwesome names without the `fa-` prefix)
    )
solvgo.on_click(solvgo_onclick)

solvlabel = widgets.Label(value='Solvate')
solvbox = widgets.VBox([solvlabel, solvgo])

emgo = widgets.Button(
        description='Minimize Energy',
        disabled=True,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to add water and counter-ions',
        icon='' # (FontAwesome names without the `fa-` prefix)
    )
emgo.on_click(emgo_onclick)

eqbox = widgets.VBox([emgo])

mdacc = widgets.Accordion(children=[],layout=widgets.Layout(width='300px'))
mdacc.set_title(0, 'Force Field')
mdacc.set_title(1, 'Solvation')
mdacc.set_title(2, 'Equilibration')
mdacc.set_title(3, 'Production')
mdacc.children = [ffbox, solvbox, eqbox]

mdreps = []

In [20]:
class amide(object): #defines amide class
    def __init__(self,C,N,H,O,isPro,CAC,CAN):
        self.C = C
        self.N = N
        self.H = H
        self.O = O
        self.isPro = isPro
        self.CAC = CAC
        self.CAN = CAN
        
angle=10

class ExcStruc(object):
    def __init__(self, cents, dips, coups, sites):
        self.nosc = np.shape(dips)[0]
        self.dips = dips
        self.coups = coups
        self.cents = cents
        self.sites = sites
        

# Locate Amide bonds
def find_bonds(struc):
    # bond_list will store an amide object for each bond in the structure
    bond_list=[]
    
    # First we loop through carbon atoms whose name is "C" and have exactly three bond partners:
    for cat in struc:
        if cat.atomic_number==6: #pulling only carbons
            if len(cat.bond_partners)==3:
                if cat.name=="C": # pulling only carbons labeled as C another solution will need to be found
                    H_id = -1 #resets H_id for each loop because this could be conditional
                    N_id = -1
                    O_id = -1
                    C_id = -1
                    CAC_id = -1
                    CAN_id = -1
                    found_N = False
                    found_CAC = False
                    found_O = False
                    NC_bonds = 0 #reset parameter to determine if proline
                    C_id = cat.idx
                    
                    bonds = cat.bond_partners
                    # We know that cat has exactly three partners. Now we check if there is exactly one 
                    # each of N, C, and O. (If not, it isn't an amide C.)
                    for n in range (0,len(bonds)): 
                        if bonds[n].atomic_number==7: #finds the nitrgoen bound to carbon C
                            N_id = bonds[n].idx
                            found_N=True

                        if bonds[n].atomic_number==6: #finds the carbon (CA) bound to carbon c
                            CAC_id = bonds[n].idx
                            found_CAC = True

                        if bonds[n].atomic_number==8: # finds the oxygen bound to cabron c
                            O_id = bonds[n].idx
                            found_O = True
                    
                    # If we pass this if(), we have an Amide bond
                    if found_N and found_CAC and found_O:
                        
                        # Now examine the bonds to the N atom: We want to find out what kind of Amide bond this is. 
                        N_bonds = struc[N_id].bond_partners
                        cat_numbers = [] # Stores indices of bonded C atoms that are NOT *the* amide C
                        at_numbers = []  # Stores indices of all bonded atoms
                        for at in N_bonds:
                            at_numbers.append(at.idx)
                            
                            if at.atomic_number==1:
                                H_id = at.idx
                                print(H_id)
                                
                            if at.atomic_number==6: #done so only carbons are recorded
                                # Don't record the Amide C atom. (We already know about it.)
                                if at.idx != C_id:
                                    cat_numbers.append(at.idx)#records the id number so that they can be recalled

                        # If cat_numbers has two entries, this is an XP bond. (Remember that Amide C index is not included.)
                        if len(cat_numbers)==2: #proline will have three carbon bonds C, delta carbon and alpha carbon
                            pro=True
                            for n in range (0,len(cat_numbers)):
                                # Examine bond partners to each C-atom bonded to the proline N atom. 
                                CAT_bonds=struc[cat_numbers[n]].bond_partners #call cat bonds
                                
                                # We distinguish between Proline CA and CD by how many C-C bonds it makes. 
                                # CD makes exactly one C-C bond (to CB). CA makes two. 
                                cneighbors=0
                                for p in range (0,len(CAT_bonds)):
                                    if CAT_bonds[p].atomic_number==6:
                                        cneighbors+=1
                                # If two C-neighbors, this is that N-terminal CA
                                if cneighbors==2:
                                    CAN_id=cat_numbers[n]
                                    
                                # If one C-neighbor, this is CD, which we record in the H_id slot. 
                                if cneighbors==1:
                                    H_id=cat_numbers[n]
                        # Non proline N atoms will have only a single (non-Amide C) C bond. 
                        if len(cat_numbers)==1:
                            pro=False
                            # This is the N-terminal CA
                            CAN_id=cat_numbers[0]

                        bond_list.append(amide(C_id,N_id,H_id,O_id,pro,CAC_id,CAN_id))
                    
    return bond_list

def find_dipoles(bond_list):
    theta = angle*(math.pi/180)#converts angle to radians

    Nbonds = len(bond_list)
    
    dip_mat = np.zeros((Nbonds,3))
    
    Cndcs = np.zeros((Nbonds,),dtype="int")
    Ondcs = np.zeros((Nbonds,),dtype="int")
    Nndcs = np.zeros((Nbonds,),dtype="int")
    Hndcs = np.zeros((Nbonds,),dtype="int")
    for n in range (0,Nbonds):
        Cndcs[n] = bond_list[n].C
        Ondcs[n] = bond_list[n].O
        Nndcs[n] = bond_list[n].N
        Hndcs[n] = bond_list[n].H

    C_coord = struc.coordinates[Cndcs]
    O_coord = struc.coordinates[Ondcs]
    N_coord = struc.coordinates[Nndcs]
    H_coord = struc.coordinates[Hndcs]
    
    CO_vec = O_coord - C_coord
    CN_vec = N_coord - C_coord
    
    
    for n in range(0, Nbonds):
        # Store CO and CN vecs for later use
        CO_vec[n,:] /= np.linalg.norm(CO_vec[n,:])
        CN_vec[n,:] /= np.linalg.norm(CN_vec[n,:])
        
        # Calculate rotation axis
        axis = np.cross(CN_vec[n,:],CO_vec[n,:])
        axis /= np.linalg.norm(axis)
        
        # Calculate rotation matrix for this bond. 
        R = spatial.transform.Rotation.from_rotvec(axis*theta)
        
        # Rotate CO vector to get dipole vector. (Should already be normalized.)
        dip_mat[n,:] = R.apply(CO_vec[n,:])/np.linalg.norm(CO_vec[n,:])
        
    # Amide bond schematic:
    # 
    #      O
    #     ||
    #     C
    #   /  \  /
    # CA    N
    #       |
    #       H
    # If C=O points up, and N is on the right, then axis points OUT OF the page. 
    # Rotation through positive angle corresponds to counter-clockwise around the axis. 
    # So positive rotations move the dipole away from N atom (which is physically correct). 
    
    cent_mat = C_coord + 0.665*CO_vec + 0.258*CN_vec
    return dip_mat, cent_mat

def calculate_coupling(dip_mat, cent_mat):
    
    # Dipole derivatives can be converted to transition dipole moments using the factor 
    # \sqrt( \hbar / 2*w_i ) = 4.1189e-25 m kg^{1/2} =  0.1011 Angstrom * amu^{1/2}
    # for a 1650 cm-1 vibration. A dipole derivative of 2.73 D/(A*amu^1/2) thus corresponds to 
    # a dipole moment matrix element of 2.73*0.1011 = 0.276 D. 
    
    dip_length = 0.276 # In Debye
    dip2 = dip_length**2
    
    Nbonds = np.shape(dip_mat)[0]
    coup_mat = np.zeros((Nbonds,Nbonds))
    
    # PREFAC should have units of Ang^3 / cm, so that when divided by 
    # the cubed inter-bond distance we obtain an energy in cm^{-1}. 
    # Starting from the squared dipole length (Debye^2):
    #    * Multiply by: (1.0e-18 statC·cm / Debye)^2 to get (statC cm)^2 = dyn cm^4 = erg cm^3
    #    * Multiply by: (1e-7 J/erg) to get J cm^3
    #    * Divide by: (6.62607e-34 J s) to get cm^3 / s 
    #    * Divide by: (2.9979e+10 cm/s) to get cm^3 / cm
    #    * Multiply by: (1e8 Ang / cm)^3 to get Ang^3 / cm
    PREFAC = dip2*((1.0e-18)**2)*(1.0e-7)*((1.0e8)**3)/( (6.62607e-34)*(2.9979e+10) )
    for m in range (0, Nbonds):
        for n in range  (0,m-1):
            R1 = cent_mat[m]
            R2 = cent_mat[n]
            r = np.linalg.norm(R2-R1)
            Rhat = (R2-R1)/r
            dip1 = dip_mat[m]
            dip2 = dip_mat[n]
            coup_mat[m,n] = PREFAC*(np.dot(dip1,dip2)-3*np.dot(dip1,Rhat)*np.dot(dip2,Rhat))/((r)**3)                        

    coup_mat +=np.transpose(coup_mat)
    specgo.disabled = False
    
    return coup_mat

In [21]:

class NGLRepList:
    def __init__(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def reset(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def append(self, nrep):
        self.names.append(nrep.name)
        ptext = {"type": nrep.type, "params": {"color": nrep.color, "sele": nrep.selection, "opacity": str(nrep.opacity)}}
        self.params.append(ptext)
        self.reps.append(nrep)
        
class NGLRep:
    def __init__(self, name, rtype, sel, col, opac):
        self.name = name
        self.type = rtype
        self.selection = sel
        self.color = col
        self.opacity = opac

def sync_widgets_to_rep(rep):
    seldrop.value = rep.name
    styledrop.value = rep.type
    colordrop.value = rep.color
    opacslide.value = int(rep.opacity*100)
    
def sync_rep_to_widgets():
    global repList
    num = repList.names.index(seldrop.value)
    rep = repList.reps[num]
    rep.type = styledrop.value
    rep.name = seldrop.value
    rep.color = colordrop.value
    rep.opacity = opacslide.value*0.01
    ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
    repList.params[num] = ptext
    pdbview.set_representations(repList.params)
    
def std_rep(pdbview, struc, bond_list):
    global repList
    
    pdbview.add_trajectory(struc)
    pdbview.clear(0)
    
    chainList = list()
    chainString = ''
    for r in struc.residues:
        if chainList.count(r.chain)==0:
            chainList.extend(r.chain)
            if len(chainString)>0:
                chainString += " OR "
            chainString += ":" + r.chain
    
    repList.append(NGLRep("Protein", "cartoon", "(protein)" + "AND (" + chainString + ")", "grey", 0.2))
    
    bond_atoms = ''
    for b in bond_list:
        if len(bond_atoms)>0:
            bond_atoms += ","
        bond_atoms += str(b.C) + "," + str(b.O)
        
    repList.append(NGLRep("Amide Bonds", "licorice", "(protein)" + "AND (@" + bond_atoms + ")", "element", 1.0))
    
    pdbview.set_representations(repList.params)
    
    seldrop.options=repList.names
    seldrop.disabled=False
    
    styledrop.disabled=False
    
    colordrop.disabled=False
    opacslide.disabled=False
    sync_widgets_to_rep(repList.reps[0])
        
pdbid = widgets.Text(
    value='1UBQ',
    placeholder='Type something',
    layout = widgets.Layout(width='2.5cm'),
    disabled=False
)

pdbidtxt = widgets.Label(value='Enter PDB ID:', width='4cm')

pdbgo = widgets.Button(
    description='Fetch',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    layout = widgets.Layout(width='2cm'),
    icon='' # (FontAwesome names without the `fa-` prefix)
)

pdbup = widgets.FileUpload(
    accept='.pdb',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

pdboutput = widgets.HTML(
    value="",
    placeholder='',
    description='',
)

pdbuptxt = widgets.Label(value='Or upload a PDB file:', width='4cm')

Output = widgets.Output()


def pdbwrite_onclick(b):
    fname = "pdb/" + accordion.children[1].children[0].children[0].value
    # First identify which chains should be displayed
    # Loop through chain-selection check-boxes
    strucList = []
    for cb in accordion.children[0].children[1:]:
        if cb.value==True:
            strucList.append(struc[cb.description,:,:])
    
    selstruc = []
    if len(strucList)>0:
        selstruc = strucList[0]
        for n in range(1, len(strucList)):
            selstruc = selstruc + strucList[n]
        selstruc.write_pdb(fname, altlocs='first')
        

    
def clear_stage(view):
    view._clear_component_auto_completion()
    if view._trajlist:
        for traj in view._trajlist:
            view._trajlist.remove(traj)
    for component_id in view._ngl_component_ids:
        component_index = view._ngl_component_ids.index(component_id)
        view._ngl_component_ids.remove(component_id)
        view._ngl_component_names.pop(component_index)
        view._remote_call('removeComponent',
            target='Stage',
            args=[component_index,])
    view._update_component_auto_completion()


def pdbup_on_value_change(change):
    for item in pdbup.value:
        fname = item
    
    with open("pdb/"+fname, "wb") as fp:
        fp.write(pdbup.value[fname]["content"])
    fp.close()
    load_structure("pdb/"+fname)
    
        
def pdbgo_onclick(b):
    fname = 'pdb/' + pdbid.value + '.pdb'
    load_structure(fname)
    
pdc_go = widgets.Button(
    description='PDC',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to calculate inter-site couplings using the Point-dipole Coupling (PDC) method',
    layout = widgets.Layout(width='2cm'),
    icon='' # (FontAwesome names without the `fa-` prefix)
)


def calc_pdc(b):
    global xstruc
    dips,cents = find_dipoles(Bonds)
    coups = calculate_coupling(dips, cents)
    xstruc = ExcStruc(cents, dips, coups, 1650.0*np.ones((1,)))
    print_output('Couplings calculated using PDC method.')

pdc_go.on_click(calc_pdc)
    
def load_structure(fname):
    global struc
    global Bonds
    
    specgo.disabled = True
    update_spec(0*vaxis)
    
    if os.path.isfile(fname):
        fid = fname.split('/')[-1]
        fid = fid.split('.')[0]
        pdbid.value=fid
        
        print_output('Local copy of file ' + pdbid.value + '.pdb already exists. Loading local structure.')
        
    else:
        url = 'http://files.rcsb.org/download/'+pdbid.value+'.pdb'
        r = requests.get(url, allow_redirects=True)
        if(r.status_code==200):
            fname = 'pdb/'+pdbid.value+'.pdb'
            wfd = open(fname, 'wb')
            wfd.write(r.content)
            wfd.close()
            pdboutput.value = ''
            print_output('Successfully downloaded PDB code ' + pdbid.value + ' from RCSB repository.')
            
        else:
            print_output('Invalid PDB code. Please try again.')
        
    if os.path.isfile(fname):
        
        while len(pdbview._ngl_component_ids)>0:
            pdbview.remove_component(pdbview._ngl_component_ids[0])
        
        accordion.children = []
        repList.reset()
        view_changes_out.clear_output(wait=False)
        
        # Read the structure
        struc = pmd.load_file(fname)
        
        # Write it to file, keeping only first set of altlocs
        struc.write_pdb(fname, altlocs='first')
        
        # Reload from single-altloc file
        struc = pmd.load_file(fname)
        
        Bonds = find_bonds(struc)
        std_rep(pdbview, struc, Bonds)
        chainList = list()
        chainString = ''
        for r in struc.residues:
            if chainList.count(r.chain)==0:
                chainList.extend(r.chain)
                chainString += r.chain
                
        chwidgList = [widgets.HBox([widgets.Button(description='All',tooltip='Click to select all chains', layout = widgets.Layout(width='1.5cm')),
                    widgets.Button(description='None',tooltip='Click to deselect all chains',layout = widgets.Layout(width='1.5cm'),)])]
        
        chwidgList[0].children[0].on_click(select_all_chains)
        chwidgList[0].children[1].on_click(deselect_all_chains)
        
        for chain in chainList:
            chwidgList.append(widgets.Checkbox(description=chain, value=True,indent=False))
            chwidgList[-1].observe(chain_box_on_change)
        accordion.children = [widgets.VBox(chwidgList)]
        
        deffnm = pdbid.value + '_' + chainString + '.pdb'
        pdbwrwidglist = [widgets.HBox([widgets.Text(value=deffnm, description='File Name:', disabled=False, layout=widgets.Layout(width='5cm')),widgets.Button(description='Write',tooltip='Click to write PDB file',layout=widgets.Layout(width='1.5cm'),)])]
        pdbwrwidglist[0].children[1].on_click(pdbwrite_onclick)
        
        accordion.children = tuple(list(accordion.children) + [widgets.VBox(pdbwrwidglist)])
        accordion.children = tuple(list(accordion.children) + [widgets.VBox([pdc_go])])
        set_md_state(False)
        solvgo.disabled = True
        emgo.disabled = True
        print_output("Successfully loaded protein structure.")
        
        #refresh_mdview()

        
    else:
        pdboutput.value = 'Please enter a valide PDB ID code.'

def select_all_chains(b):
    for cb in accordion.children[0].children[1:]:
        cb.value = True
        
def deselect_all_chains(b):
    for cb in accordion.children[0].children[1:]:
        cb.value = False
        
        
def chain_box_on_change(b):
    global repList
    if b['type'] == 'change' and b['name'] == 'value':
        
        # First identify which chains should be displayed
        chainString = ''
        chainList = []
        chainChars = ''
        # Loop through chain-selection check-boxes
        for cb in accordion.children[0].children[1:]:
            if cb.value==True:
                chainList.extend(cb.description)
                if len(chainString)>0:
                    chainString += " OR "
                chainString += ":" + cb.description
                chainChars += cb.description
                
        for num in range(0, len(repList.reps)):
            rep = repList.reps[num]
            selList = rep.selection.split('AND')
            selText = ''
            for item in selList:
                if item.find(":")==-1:
                    if len(selText)>0:
                        selText += " AND "
                    selText += item.strip()
            
            if len(chainString)>0:
                selText += " AND (" + chainString + ")"
            else:
                # We assume no structure has chain ID XXXX...
                selText += " AND (:XXXX)"
                
            rep.selection = selText
            ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
            repList.params[num] = ptext
        pdbview.set_representations(repList.params)
        
        if len(chainString)>0:
            pdbview.set_representations([{"type": "licorice", "params": {"color": "red", "sele": "("+chainString+")", "opacity": "1", "radius": "0.35"}}], component=1)
            accordion.children[1].children[0].children[0].value = pdbid.value + "_" + chainChars + '.pdb'
            accordion.children[1].children[0].children[1].disabled = False
            accordion.children[2].children[0].disabled = False
            set_md_state(False)
        else:
            pdbview.set_representations([{"type": "licorice", "params": {"color": "red", "sele": "(:XXXX)", "opacity": "1", "radius": "0.35"}}], component=1)
            accordion.children[1].children[0].children[0].value = ''
            accordion.children[1].children[0].children[1].disabled = True
            accordion.children[2].children[0].disabled = True
            set_md_state(True)
            
def seldrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        rep = repList.reps[repList.names.index(seldrop.value)]
        sync_widgets_to_rep(rep)
        
def styledrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = repList.names.index(seldrop.value)
        rep = repList.reps[num]
        rep.type = styledrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        repList.params[num] = ptext
        pdbview.set_representations(repList.params)
        
def colordrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = repList.names.index(seldrop.value)
        rep = repList.reps[num]
        rep.color = colordrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        repList.params[num] = ptext
        pdbview.set_representations(repList.params)
        
def opacslide_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = repList.names.index(seldrop.value)
        rep = repList.reps[num]
        rep.opacity = opacslide.value*0.01
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        repList.params[num] = ptext
        pdbview.set_representations(repList.params)

repList = NGLRepList()
seldrop = widgets.Dropdown(
    options=repList.names,
    #value='Protein',
    description='Selection:',
    disabled=True,
)

styledrop = widgets.Dropdown(
    options=['cartoon', 'licorice', 'spacefill'],
    description='Style:',
    disabled=True,
)

colordrop = widgets.Dropdown(
    options=['chain', 'red', 'green', 'blue', 'grey'],
    description='Color:',
    disabled=True,
)

opacslide = widgets.IntSlider(
    min=0,
    max=100,
    step=1,
    description='Opacity:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

seldrop.observe(seldrop_on_change)
styledrop.observe(styledrop_on_change)
colordrop.observe(colordrop_on_change)
opacslide.observe(opacslide_on_change)


accordion = widgets.Accordion(children=[])
accordion.set_title(0, 'Chain Selection')
accordion.set_title(1, 'Write PDB')
accordion.set_title(2, 'Calculate Couplings')
accordion.set_title(3, 'Prepare for MD')

#pdbuptext = widgets.Label(value="Or upload a file:", layout=widgets.Layout(width='3.5cm', display="flex", justify_content='flex-end'))

pdbgo.on_click(pdbgo_onclick)

pdbup.observe(pdbup_on_value_change, 'value')
pdbid.on_submit(pdbgo_onclick)


abscb = widgets.Checkbox(
    value=True,
    description='Absorption',
    disabled=False
)

# cdcb = widgets.Checkbox(
#     value=False,
#     description='CD',
#     disabled=False
# )

def get_spec_lims(lines):
    
    myinf = 1e+100
    mny = myinf
    mxy = -myinf
    for line in lines:
        if line.get_visible()==True:
            mny = min(mny, np.min(line.get_ydata()))
            mxy = max(mxy, np.max(line.get_ydata()))
    
    mnx = len(vaxis) - 1
    mxx = 0
    cutoff = 1e-3
    for line in lines:
        if line.get_visible()==True:
            ndcs = np.where(np.abs(line.get_ydata())>cutoff*mxy)[0]
            if len(ndcs>0):
                mnx = min(mnx, np.min(ndcs))
                mxx = max(mxx, np.max(ndcs))
    return mny, mxy, vaxis[mnx], vaxis[mxx]
    

# def update_spec_visibility(b):
#     if abscb.value==True:
#         abs_line.set_visible(True)
#     else:
#         abs_line.set_visible(False)
        
#     update_spec_lims()
        
# abscb.observe(update_spec_visibility)
# cdcb.observe(update_spec_visibility)

def specgo_onclick(b):
    
    Nosc = xstruc.nosc
    
    if (np.shape(xstruc.coups)[0]!=Nosc) or (np.shape(xstruc.coups)[1]!=Nosc):
        print('Please calculate coupling constants before calculate spectra.')
        return 0
    else:
        
        abs_spec = np.zeros(np.shape(vaxis))
        if len(xstruc.sites)==1:
            Ham = xstruc.coups + np.eye(Nosc)*xstruc.sites
        else:
            Ham = xstruc.coups + np.diag(xstruc.sites)
        eVals,eVecs = np.linalg.eigh(Ham)
        eMu = np.matmul(np.transpose(eVecs), xstruc.dips)

        for n in range(0,Nosc):
            ndx = np.where(np.abs(vaxis-eVals[n])==np.min(np.abs(vaxis-eVals[n])))[0][0]
            abs_spec[ndx] += np.linalg.norm(eMu[n])**2/Nosc #mu is a 3 element vector

        abs_spec = gaussian_filter(abs_spec, sigma=5)
        update_spec(abs_spec)


specview = widgets.Output()
specgo = widgets.Button(description='Calculate', disabled=True)
specgo.on_click(specgo_onclick)
spec_checks = widgets.VBox([specgo, abscb])
specbox = widgets.HBox([spec_checks, specview])

pdbview = nv.NGLWidget()
pdbview._set_size('400px', '400px')

# mdview = nv.NGLWidget()
# mdview._set_size('400px', '400px')
# mdviewout = widgets.Output()

pdbbox = widgets.VBox([widgets.HBox([pdbidtxt, pdbid, pdbgo]), widgets.HBox([pdbuptxt, pdbup])])
dispbox = widgets.VBox([pdbbox, seldrop, styledrop, colordrop, opacslide, accordion])
strucbox = widgets.HBox([dispbox, pdbview])

# mdbox = widgets.HBox([widgets.VBox([mdacc]), mdview])
mdbox = widgets.HBox([widgets.VBox([mdacc])])

maintab = widgets.Tab([strucbox, specbox, mdbox])
maintab.set_title(0, 'Structure')
maintab.set_title(1, 'Spectrum')
maintab.set_title(2, 'Molecular Dynamics')
maintab.selected_index = 0

def print_output(text):
#     global OutTxt
    
# #     MaxLines = 4
# #     if len(OutTxt)>MaxLines:
# #         del OutTxt[0]
#     OutTxt.append(text)
        
#     mainout.clear_output(wait=False)
#     with mainout:
#         for line in OutTxt[-1::-1]:
#             print(line)
            
    with mainout:
        print(text)
    print_log([text])
        
def print_log(output):
    with logout:
        for line in output:
            print(line)
        
mainout = widgets.Output(layout=widgets.Layout(height='120px', overflow='scroll'))
logout = widgets.Output(layout=widgets.Layout(height='500px', overflow='scroll'))

display(pdboutput)
display(maintab)
# display(mainout)
outacc = widgets.Accordion(children=[mainout,logout])
outacc.set_title(0, 'Output')
outacc.set_title(1, 'Log')
display(outacc)
# with mdviewout:
#     display(mdview)

def format_coord(x, y):
    return 'x=%1.1f, y=%1.2f'%(x, y)

vaxis = np.arange(1400, 1800)
#vaxis = np.arange(14000, 14000.1, 0.1)

def update_spec(data):
    global abs_line
    with specview:
        %matplotlib widget
        fig = plt.figure("Calculated Spectra", figsize=(4,4))
        abs_line, = plt.plot(vaxis, data)
        plt.plot(vaxis, 0*vaxis, 'k--')
        specax = plt.gca()
        specax.set_xlabel('$\\omega/2\\pi$c (cm$^{-1}$)')
        specax.set_ylabel('Signal (arbitrary units)')
        specax.set_yticklabels([])
        specax.format_coord = format_coord
        mny, mxy, mnx, mxx = get_spec_lims([abs_line])
        if mny<mxy:
            gap = mxy - mny
            plt.ylim(mny-0.1*gap, mxy+0.1*gap)
        if mnx<mxx:
            plt.xlim(mnx, mxx)
            
        plt.tight_layout()
    
struc = pmd.structure.Structure()
xstruc = []
Bonds = []
update_spec(0*vaxis)

OutTxt = []


HTML(value='', placeholder='')

Accordion(children=(Output(layout=Layout(height='120px', overflow='scroll')), Output(layout=Layout(height='500…